# Setup

Initial module setup.

In [1]:
import logging
import numpy.typing as np_types
import pandas as pd

from concrete.ml.sklearn.rf import RandomForestClassifier
from eeg_auth_models_framework import data, pre_process, features, training, model, processor
from eeg_auth_models_framework.utils import conversion

# Constants

In [2]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10
RANDOM_SEED = 42

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [3]:
class MusicIDModelBuilder(model.ModelBuilder[RandomForestClassifier]):
    def create_classifier(self) -> RandomForestClassifier:
        return RandomForestClassifier(
            n_estimators=100,
            criterion='gini',
            max_depth=10,
            random_state=RANDOM_SEED
        )
    
    def train_classifier(self, classifier: RandomForestClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: RandomForestClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Configuration

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
labeller = training.SubjectDataLabeller()
converter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)

# Data Processing

## Pre-Processing Steps

Define pre-processing steps to be used in model.

In [5]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES,
        converter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

## Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [6]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.StatisticalFeatureExtractor([
        features.StatisticalFeature.MIN,
        features.StatisticalFeature.MAX,
        features.StatisticalFeature.MEAN,
        features.StatisticalFeature.ZERO_CROSSING_RATE
    ])
])

## Data Processor

In [7]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Training

Execute training of authentication models.

In [8]:
music_id_builder = MusicIDModelBuilder(
    data_downloader=downloader,
    data_reader=reader,
    data_labeller=labeller,
    data_processor=data_processor
)
results = music_id_builder.train(K_FOLDS)

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

In [9]:
subject_score_rows = []
for subject in results.training_statistics:
    subject_score_rows.append([
        subject,
        results.training_statistics[subject].average_score,
        results.training_statistics[subject].training_duration
    ])
subject_scores = pd.DataFrame(subject_score_rows, columns=['Subject', 'Average Score', 'Training Duration'])
subject_scores

,Subject,Average Score,Training Duration
0,S13,1.000000,4.419042
1,S14,1.000000,4.626948
2,S15,1.000000,4.701381
3,S11,1.000000,4.296983
4,S17,0.998765,3.710124
5,S07,1.000000,4.388154
6,S04,1.000000,4.367440
7,S18,1.000000,3.768438
8,S19,1.000000,4.451507
9,S05,1.000000,4.415598


In [10]:
subject_rates_rows = []
for subject in results.training_statistics:
    subject_rates_rows.append([
        subject,
        results.training_statistics[subject].average_false_accept_rate,
        results.training_statistics[subject].average_false_reject_rate
    ])
subject_rates = pd.DataFrame(subject_rates_rows, columns=['Subject', 'Average FAR', 'Average FRR'])
subject_rates

,Subject,Average FAR,Average FRR
0,S13,0.000000,0.000
1,S14,0.000000,0.000
2,S15,0.000000,0.000
3,S11,0.000000,0.000
4,S17,0.000000,0.025
5,S07,0.000000,0.000
6,S04,0.000000,0.000
7,S18,0.000000,0.000
8,S19,0.000000,0.000
9,S05,0.000000,0.000


In [11]:
global_stats = pd.DataFrame(
    [[results.global_average_score, results.global_average_far_rate, 
      results.global_average_frr_rate, results.global_average_time]],
    columns=['Global Average Score', 'Global Average FAR', 'Global Average FRR', 'Global Average Time']
)
global_stats

,Global Average Score,Global Average FAR,Global Average FRR,Global Average Time
0,0.99284,0.00375,0.007306,4.169182
